# Lesson 4: Auto-merging Retrieval

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["Main_Data.pdf"]
).load_data()

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

280 

<class 'llama_index.schema.Document'>
Doc ID: 755034d6-22b2-46d8-9569-96b825dc28f5
Text: Title: " Car Reviews"   Subject: “ Youtube Reviews and Detailed
analysis of different  car models"     Car Model  :  2023 Mini
Countryman   Review :    car enthusiasts we've all had our fair share
of positive and negative experiences with car enthusiasts  that's the
thing with owners they're always passionate about what they drive and
and someti...


## Auto-merging retrieval setup

In [5]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [7]:
nodes = node_parser.get_nodes_from_documents([document])

In [8]:
from llama_index.node_parser import get_leaf_nodes

leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

to 
add a untamed edition which that package will add  gray metallic paint with piano black exterior 
highlights on the s and the hybrid trim inside the funda drive suv receives updated tech sports seats the 
larger cooper goes against rivals like your mazda cx -30 your volvo xc40 even the honda hrv that's been 
redesigned but is this one going to be more


In [9]:
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

to unstick you and it'll also kind of just balance out between the front and rear axle the 
power delivery so it gives you the best possible chance of tackling things like ice and snow of course I 
haven't been able to ha ve a chance to actually tell how this works but you can tell that by the way it 
sticks around corners the way that it handles then it will be pretty robust but of course you can't go 
comparing it to something like a Land Rover Defender despite what the nam e might suggest and the 
addition of all four the mini Countryman Untamed isn't exactly an off -roader but then again anyone that 
knows their minis knows that these cars were not really designed for that they're designed for 
Adventures perhaps more adventure s in a city and it's really good at doing that I've had a fantastic time 
with the mini Countryman untamed and I think that interior is really really very special it's made me fall 
in love with this car I think I've said that three times now but it's true t

### Building the index

In [10]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [11]:
from llama_index import ServiceContext

auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

In [12]:
from llama_index import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [13]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


### Defining the retriever and running the query engine

In [14]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=25
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

rerank = SentenceTransformerRerank(top_n=5, model="BAAI/bge-reranker-base")

auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [15]:
auto_merging_response = auto_merging_engine.query(
    "What are some unique features of Jaguar cars?"
)

In [16]:
from llama_index.response.notebook_utils import display_response

display_response(auto_merging_response)

**`Final Response:`** Jaguar cars have unique features such as sleek vents, a dished three-spoke wheel, a tactile gear lever, sophisticated media connectivity, authentic finishes in beautifully formed shapes, a redesigned center console, metal finishing on the gear stick, steering wheel, and door pulls, and a mesh modeled in the shape of the Jaguar heritage logo. Additionally, Jaguar cars have slimmer headlights with a new design, LED running lights, and redesigned taillights.

## Putting it all Together

In [46]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=20,
    rerank_top_n=10,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [47]:
from llama_index.llms import OpenAI

index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)


In [48]:
query_engine = get_automerging_query_engine(index, similarity_top_k=10)

## TruLens Evaluation

In [49]:
from trulens_eval import Tru

Tru().reset_database()

### Two layers

In [50]:
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512,128],
)

In [51]:
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=50,
    rerank_top_n=10,
)

In [52]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'
)

In [53]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [54]:
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [55]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ed43c71270 is calling an instrumented method <function BaseQueryEngine.query at 0x000001EBF119ADD0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1ebb6338100) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ed43c71270 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001EBFA644040>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1ebb6338100) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x1ed627e4b20 is calling an instrumented method <function BaseRetriever.retrieve at 0x000001EBF119A170>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obje

> Merging 2 nodes into parent node.
> Parent node id: c52cf69c-2c0c-446f-b30f-68924226ed39.
> Parent node text: structured properly so overall dashboard design is obviously very attractive you get a heads -up ...

> Merging 2 nodes into parent node.
> Parent node id: d9f9f79f-0adb-4de8-aad8-ed21031fedee.
> Parent node text: and the gearbox and the drivetrain just feel so familiar to me so similar to the clubman but it's...

> Merging 2 nodes into parent node.
> Parent node id: 4423a71d-0bc9-4f45-83a6-6aea1ff83017.
> Parent node text: been basically carried over from BMW the infotainment system is a previous generation of iDrive 
...

> Merging 3 nodes into parent node.
> Parent node id: 27c86d16-0357-4cad-9912-7ea79fe826ce.
> Parent node text: interior of the mini feels very very open very very spacious which is kind of odd given that thes...

> Merging 2 nodes into parent node.
> Parent node id: ee315fda-d9ee-4d77-9a63-695302243f32.
> Parent node text: to let me use one of their bmw x

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1ed43c72b90 is calling an instrumented method <function CompactAndRefine.get_response at 0x000001EBF2ED8280>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1ebb6338820) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1ed43c72b90 is calling an instrumented method <function Refine.get_response at 0x000001EBF36D0EE0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1ebb6338820) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x1ec04373f40 is calling an instrumented method <function LLMPredictor.predict at 0x000001EBEFFFAB00>. The path of this call may be incorrect.
Guessing path of new object is app._response_

> Merging 2 nodes into parent node.
> Parent node id: 7e050d77-c6da-44ae-a81c-72339f558073.
> Parent node text: the same as the x2 m35i i think the mini is still far more fun to drive but i'm gonna have to say...

> Merging 2 nodes into parent node.
> Parent node id: e86edd1c-3b94-4dc6-821c-37d428dd67b9.
> Parent node text: Title: "Comprehensive Car Reviews 2023"  
Subject: "Expert reviews and detailed analysis of 2023 ...

> Merging 3 nodes into parent node.
> Parent node id: f9f9c32e-f470-4663-9312-cab9c747bd58.
> Parent node text: back here as well I am obsessed with this leather interior it's so pretty I love the contrasting ...

> Merging 2 nodes into parent node.
> Parent node id: ee315fda-d9ee-4d77-9a63-695302243f32.
> Parent node text: to let me use one of their bmw x2s so there it is there's the countryman in question and this is ...

> Merging 2 nodes into parent node.
> Parent node id: 25783423-fd37-49b7-8896-38bd2ea612fd.
> Parent node text: having one up front on your hood

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1ed43c72b90 is calling an instrumented method <function Refine.get_response at 0x000001EBF36D0EE0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1ebb6338820) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ed43c71270 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001EBFA644040>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1ebb6338100) using this function.


...

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1ed43c72b90 is calling an instrumented method <function Refine.get_response at 0x000001EBF36D0EE0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1ebb6338820) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ed43c71270 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001EBFA644040>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1ebb6338100) using this function.


> Merging 2 nodes into parent node.
> Parent node id: a2b15458-54ff-4bef-8913-b19724a15a36.
> Parent node text: fast with shifts only thing is sometimes it doesn't give you the downshift which you're looking f...

> Merging 2 nodes into parent node.
> Parent node id: d51689ef-add5-4664-b152-422279a02825.
> Parent node text: it [Music] minnie has been getting a lot of flack and loyalists are right to say that it's not th...

> Merging 2 nodes into parent node.
> Parent node id: d9f9f79f-0adb-4de8-aad8-ed21031fedee.
> Parent node text: and the gearbox and the drivetrain just feel so familiar to me so similar to the clubman but it's...

> Merging 2 nodes into parent node.
> Parent node id: 6fe75a8e-2497-4e8a-b67c-fa1443d15777.
> Parent node text: automatic gearbox now let's see what this mini is like to drive starting off with what it's like ...

> Merging 3 nodes into parent node.
> Parent node id: 27c86d16-0357-4cad-9912-7ea79fe826ce.
> Parent node text: interior of the mini feels very 

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1ed43c72b90 is calling an instrumented method <function Refine.get_response at 0x000001EBF36D0EE0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1ebb6338820) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x1ed43c71270 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x000001EBFA644040>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x1ebb6338100) using this function.


> Merging 2 nodes into parent node.
> Parent node id: 4423a71d-0bc9-4f45-83a6-6aea1ff83017.
> Parent node text: been basically carried over from BMW the infotainment system is a previous generation of iDrive 
...

> Merging 3 nodes into parent node.
> Parent node id: f9f9c32e-f470-4663-9312-cab9c747bd58.
> Parent node text: back here as well I am obsessed with this leather interior it's so pretty I love the contrasting ...

> Merging 2 nodes into parent node.
> Parent node id: d9f9f79f-0adb-4de8-aad8-ed21031fedee.
> Parent node text: and the gearbox and the drivetrain just feel so familiar to me so similar to the clubman but it's...

> Merging 2 nodes into parent node.
> Parent node id: 7e050d77-c6da-44ae-a81c-72339f558073.
> Parent node text: the same as the x2 m35i i think the mini is still far more fun to drive but i'm gonna have to say...

> Merging 2 nodes into parent node.
> Parent node id: 47640b78-78eb-4be5-952a-29acafa71478.
> Parent node text: wheel you're like a nimble easy 

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x1ed43c72b90 is calling an instrumented method <function Refine.get_response at 0x000001EBF36D0EE0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x1ebb6338820) using this function.


In [58]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.061,0.9,0.516667,44.6,0.012955


In [59]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.142.162:8503



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
gfgf

### Three layers

In [37]:
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048,512,128],
)

In [38]:
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=10,
    rerank_top_n=6,
)


In [39]:
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tru_recorder = get_prebuilt_trulens_recorder(                                                │
│   2 │   auto_merging_engine_1,                                                                   │
│   3 │   app_id ='app_1'                                                                          │
│   4 )                                                                                            │
│                                                                                                  │
│ c:\Users\enesm\OneDrive\Masaüstü\LLAMAINDEX\utils.py:68 in get_prebuilt_trulens_recorder         │
│                                                                                                  │
│    65 │   │   query_engine,                                                                      │
│    66 │   │   app_id=app_id,                                                                     │
│    67 │   │   feedbacks=feedbacks                                                                │
│ ❱  68 │   │   )                                                                                  │
│    69 │   return tru_recorder                                                                    │
│    70                                                                                            │
│    71 from llama_index import ServiceContext, VectorStoreIndex, StorageContext                   │
│                                                                                                  │
│ c:\Users\enesm\AppData\Local\Programs\Python\Python310\lib\site-packages\trulens_eval\tru_llama. │
│ py:250 in __init__                                                                               │
│                                                                                                  │
│   247 │   │                                                                                      │
│   248 │   │   super().__init__(**kwargs)                                                         │
│   249 │   │                                                                                      │
│ ❱ 250 │   │   self.post_init()                                                                   │
│   251 │                                                                                          │
│   252 │   @classmethod                                                                           │
│   253 │   def select_source_nodes(cls) -> JSONPath:                                              │
│                                                                                                  │
│ c:\Users\enesm\AppData\Local\Programs\Python\Python310\lib\site-packages\trulens_eval\app.py:486 │
│ in post_init                                                                                     │
│                                                                                                  │
│    483 │   │   │   │   logger.debug("Inserting feedback function definitions to db.")            │
│    484 │   │   │   │                                                                             │
│    485 │   │   │   │   for f in self.feedbacks:                                                  │
│ ❱  486 │   │   │   │   │   self.db.insert_feedback_definition(f)                                 │
│    487 │   │                                                                                     │
│    488 │   │   else:                                                                             │
│    489 │   │   │   if len(self.feedbacks) > 0:                                                   │
│                                                            

In [40]:
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

> Merging 2 nodes into parent node.
> Parent node id: 2ab9ba8f-6ecb-4d33-a74f-bc7dcf52ee05.
> Parent node text: it would not I wouldn't h ave 
done myself Justice if I didn't you know at least look at those an...

> Merging 1 nodes into parent node.
> Parent node id: 87be3bbe-b0f6-464e-b6f2-ec96d8ea30ae.
> Parent node text: the bmw x1 as well which is technic ally engineers under the 
bmw umbrella today we're going to t...

> Merging 2 nodes into parent node.
> Parent node id: e3fcc075-9d17-45bf-812a-8dc64b8fc589.
> Parent node text: it gives you quite a bit of wheel spin and off we 
go so performance is actually very very very n...

> Merging 2 nodes into parent node.
> Parent node id: d631dd59-0c79-4566-af67-3053292552b4.
> Parent node text: talking about 
uncertainty because of the expansion joint on the expansion like oh my goodness th...



In [41]:
from trulens_eval import Tru

Tru().get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
app_0,0.060833,0.885714,0.629688,35.4,0.009249


In [42]:
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.142.162:8503



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>